In [1]:
import numpy as np
from sympy import *
init_printing(use_latex='mathjax')

In [2]:
def get_jacobian(expressions, symbols):
    rows = len(expressions)
    columns = len(symbols)

#     print("Expressions:")
#     for expression in expressions:
#         display(expression)

    results = [[0 for x in range(columns)] for y in range(rows)] 
    for row, expression in enumerate(expressions):
        for column, symbol in enumerate(symbols):
#             print('Row %d, column %d, expression: %s, symbol: %s' % (row, column, expression, symbol))
            df = diff(expression, symbol)
#             print("DF: %s" % df)
            results[row][column] = df
    return results

In [3]:
def get_hessians(jacobian, symbols):
    jacobian = np.array(jacobian)
    rows, columns = jacobian.shape
    hessians = [[[None for z in range(len(symbols))] for x in range(columns)] for y in range(rows)] 
    for row in range(rows):
        for column in range(columns):
            for index, symbol in enumerate(symbols):
                df = diff(jacobian[row, column], symbol)
                hessians[row][column][index] = df
            
    return hessians

In [14]:
def evaluate_at(exprs, symbols):
    results = []
    def process_list(exprs, symbols):
        r = []
        for expr in exprs:
            if isinstance(expr, list):
                r.append(process_list(expr, symbols))
            else:
                r.append(expr.subs(symbols))
        return r

    results = []
    if isinstance(exprs, list):
        results.append(process_list(exprs, symbols))
    else:
        results.append(exprs.subs(symbols))
    return results

In [16]:
x, y,z = symbols('x y z')
s = [x, y, z]
expr = x ** 2 * cos(y) + exp(z) * sin(y)
j = get_jacobian([expr], s)
r = evaluate_at(j, {x: pi, y: pi, z: 1})
r

Expressions:


 2           z       
x ⋅cos(y) + ℯ ⋅sin(y)

[[[-2⋅π, -ℯ, 0]]]

In [17]:
x, y = symbols('x y')
s = [x, y]
j = get_jacobian([
    x**2 * y - cos(x) * sin(y),
    exp(x + y)
], s)
r = evaluate_at(j, {x: 0, y: pi})
r

Expressions:


 2                  
x ⋅y - sin(y)⋅cos(x)

 x + y
ℯ     

⎡⎡        ⎡ π   π⎤⎤⎤
⎣⎣[0, 1], ⎣ℯ , ℯ ⎦⎦⎦

In [19]:
x, y = symbols('x y')
s = [x, y]
j = get_jacobian([
    x ** 3 * cos(y) - x * sin(y)
], s)
h = get_hessians(j, s)
h

Expressions:


 3                  
x ⋅cos(y) - x⋅sin(y)

⎡⎡⎡                 2                ⎤  ⎡     2                     3         
⎣⎣⎣6⋅x⋅cos(y), - 3⋅x ⋅sin(y) - cos(y)⎦, ⎣- 3⋅x ⋅sin(y) - cos(y), - x ⋅cos(y) +

         ⎤⎤⎤
 x⋅sin(y)⎦⎦⎦

In [20]:
x, y, z = symbols('x y z')
s = [x, y, z]
j = get_jacobian([
    x * y + sin(y) * sin(z) + z ** 3 * exp(x)
], s)
h = get_hessians(j, s)
h

Expressions:


       3  x                
x⋅y + z ⋅ℯ  + sin(y)⋅sin(z)

⎡⎡⎡ 3  x        2  x⎤                                      ⎡   2  x           
⎣⎣⎣z ⋅ℯ , 1, 3⋅z ⋅ℯ ⎦, [1, -sin(y)⋅sin(z), cos(y)⋅cos(z)], ⎣3⋅z ⋅ℯ , cos(y)⋅co

           x                ⎤⎤⎤
s(z), 6⋅z⋅ℯ  - sin(y)⋅sin(z)⎦⎦⎦

In [22]:
x, y, z = symbols('x y z')
s = [x, y, z]
j = get_jacobian([
    x * y *  cos(z) - sin(x) * exp(y) * z ** 3
], s)
h = get_hessians(j, s)
evaluate_at(h, {x: 0, y: 0, z: 0 })

Expressions:


              3  y       
x⋅y⋅cos(z) - z ⋅ℯ ⋅sin(x)

[[[[0, 1, 0], [1, 0, 0], [0, 0, 0]]]]